This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string from a config file- according to PEP8, constants are always named in all upper case
import configparser

config = configparser.ConfigParser()
config.read('API_KEYS.ini')
config.sections()

API_KEY = config['quandl.com']['API_KEY']

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests


In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
API_CALL = "https://www.quandl.com/api/v3/datasets/WIKI/FB.json?column_index=4&start_date=2014-01-15&end_date=2014-01-16&collapse=daily&transform=rdiff&api_key="

r = requests.get(API_CALL + API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset': {'id': 9775687, 'dataset_code': 'FB', 'database_code': 'WIKI', 'name': 'Facebook Inc. (FB) Prices, Dividends, Splits and Trading Volume', 'description': "End of day open, high, low, close and volume, dividends and splits, and split/dividend adjusted open, high, low close and volume for Facebook, Inc. (FB). Ex-Dividend is non-zero on ex-dividend dates. Split Ratio is 1 on non-split dates. Adjusted prices are calculated per CRSP (www.crsp.com/products/documentation/crsp-calculations)\n\nThis data is in the public domain. You may copy, distribute, disseminate or include the data in other products for commercial and/or noncommercial purposes.\n\nThis data is part of Quandl's Wiki initiative to get financial data permanently into the public domain. Quandl relies on users like you to flag errors and provide data where data is wrong or missing. Get involved: connect@quandl.com\n", 'refreshed_at': '2018-03-27T21:46:11.036Z', 'newest_available_date': '2018-03-27', 'oldest_available_

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#Call to get all columns
API_CALL_ALL = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?start_date=2017-01-01&end_date=2017-12-31&collapse=daily&api_key="
r = requests.get(API_CALL_ALL + API_KEY)

#Call to get closing price, rdiff transformed
API_CALL_DIFF = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?column_index=4&start_date=2017-01-01&end_date=2017-12-31&collapse=daily&transform=diff&api_key="
r_diff = requests.get(API_CALL_DIFF + API_KEY)

In [6]:
#dictionary of all values
afx_x = r.json()['dataset']['data']
afx_x = {d[0]: d[1:] for d in afx_x}

#dictionary of diffference in open prices
afx_x_diff = r_diff.json()['dataset']['data']
afx_x_diff = {d[0]: d[1] for d in afx_x_diff}

In [7]:
dates = []
opens = []
day_deltas = []
volumes = []
for date, values in afx_x.items():
    dates.append(date)
    opens.append(values[0])
    volumes.append(values[6])
    day_deltas.append(values[1] - values[2])


diff_dates = []
close_deltas = []
for date, value in afx_x_diff.items():
    diff_dates.append(date)
    close_deltas.append(abs(value))


In [8]:
#get index of lowest open
low_index = opens.index(min(open_val for open_val in opens if open_val is not None))
#get index of highest open
hi_index = opens.index(max(open_val for open_val in opens if open_val is not None))

ans_3a = ('%s had lowest opening: %s' % (dates[low_index], opens[low_index]))
ans_3b = ('%s had highest opening: %s' % (dates[hi_index], opens[hi_index]))

In [9]:
#get highest within-day change
hi_day_delta_index = day_deltas.index(max(delta_val for delta_val in day_deltas if delta_val is not None))

ans_4 = ('%s had greatest within day difference: %s' % (dates[hi_day_delta_index], day_deltas[hi_day_delta_index]))

In [10]:
#get highest interday change
hi_iday_delta_index = close_deltas.index(max(close_delta for close_delta in close_deltas if close_delta is not None))

ans_5 = ('%s had greatest between-day difference: %s' % (dates[hi_iday_delta_index], close_deltas[hi_iday_delta_index]))

In [11]:
#get average trading volume
ans_6 = ('average trading volume: %s' % (sum(volumes)/len(volumes)) )

In [12]:
#get median trading volume
def median(lst):
    n = len(lst)
    if n < 1:
            return None
    if n % 2 == 1:
            return sorted(lst)[n//2]
    else:
            return sum(sorted(lst)[n//2-1:n//2+1])/2.0

median(volumes)
ans_7 = ('median trading volume: %s' % median(volumes))

In [13]:
#print answers

print('3. Calculate what the highest and lowest opening prices were for the stock in this period \n')
print(ans_3a)
print(ans_3b,'\n')

print('4. What was the largest change in any one day (based on High and Low price)? \n')
print(ans_4,'\n')

print('5. What was the largest change between any two days (based on Closing Price)? \n')
print(ans_5,'\n')

print('6. What was the average daily trading volume during this year? \n')
print(ans_6,'\n')

print('7. What was the median trading volume during this year. (Optional) \n') 
print(ans_7,'\n')

3. Calculate what the highest and lowest opening prices were for the stock in this period 

2017-01-24 had lowest opening: 34.0
2017-12-14 had highest opening: 53.11 

4. What was the largest change in any one day (based on High and Low price)? 

2017-05-11 had greatest within day difference: 2.8100000000000023 

5. What was the largest change between any two days (based on Closing Price)? 

2017-08-09 had greatest between-day difference: 2.56 

6. What was the average daily trading volume during this year? 

average trading volume: 3853589.4509803923 

7. What was the median trading volume during this year. (Optional) 

median trading volume: 3292223.0 

